In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

try:
    import numpy as np
except ImportError:
    raise ImportError("NumPy is not installed.")

class HungarianError(Exception):
    pass

class Hungarian:
    def __init__(self):
        self._cost_matrix = None
        self._input_matrix = None
        self._maxColumn = 0
        self._maxRow = 0
        self._results = []
        self._totalPotential = 0

    def input_matrix(self, input_matrix, is_profit_matrix=False):
        """
        Accepts a matrix input and determines whether it is a cost or profit matrix.
        """
        # Save input
        my_matrix = np.array(input_matrix)
        self._input_matrix = my_matrix
        self._maxColumn = my_matrix.shape[1]
        self._maxRow = my_matrix.shape[0]

        # Padding
        matrix_size = max(self._maxColumn, self._maxRow)
        pad_columns = matrix_size - self._maxRow
        pad_rows = matrix_size - self._maxColumn
        my_matrix = np.pad(my_matrix, ((0, pad_columns), (0, pad_rows)), 'constant', constant_values=(0))

        # Convert matrix to cost matrix if necessary
        if is_profit_matrix:
            my_matrix = self.make_cost_matrix(my_matrix)

        self._cost_matrix = my_matrix

    def calculate(self):
      if self._cost_matrix is None:
        raise HungarianError("No matrix input provided.")
      result_matrix = self._cost_matrix.copy()

    # Step 1: Subtract row mins from each row
      for index, row in enumerate(result_matrix):
          result_matrix[index] -= row.min()
      print("After subtracting row minimums:\n", result_matrix)  # Added print statement

    # Step 2: Subtract column mins from each column
      for index, column in enumerate(result_matrix.T):
          result_matrix[:, index] -= column.min()
      print("After subtracting column minimums:\n", result_matrix)  # Added print statement

    # Step 3: Cover zeros
      total_covered = 0
      while total_covered < self._maxRow:
          cover_zeros = CoverZeros(result_matrix)
          covered_rows = cover_zeros.get_covered_rows()
          covered_columns = cover_zeros.get_covered_columns()
          total_covered = len(covered_rows) + len(covered_columns)

          print("Covered rows:", covered_rows)  # Added print statement
          print("Covered columns:", covered_columns)  # Added print statement

          if total_covered < self._maxRow:
              result_matrix = self._adjust_matrix_by_min_uncovered_num(result_matrix, covered_rows, covered_columns)
              print("After adjusting matrix by min uncovered number:\n", result_matrix)  # Added print statement

      expected_results = min(self._maxColumn, self._maxRow)
      zero_locations = (result_matrix == 0)
      while len(self._results) != expected_results:
          if not zero_locations.any():
              raise HungarianError("Unable to find results. Algorithm has failed.")

          matched_rows, matched_columns = self.__find_matches(zero_locations)

          total_matched = len(matched_rows) + len(matched_columns)
          if total_matched == 0:
              matched_rows, matched_columns = self.select_arbitrary_match(zero_locations)

          for row in matched_rows:
              zero_locations[row] = False
          for column in matched_columns:
              zero_locations[:, column] = False

          self.__set_results(zip(matched_rows, matched_columns))

    # Calculate total potential
      self._totalPotential = sum(self._input_matrix[row, column] for row, column in self._results)

    @staticmethod
    def make_cost_matrix(profit_matrix):
        matrix_shape = profit_matrix.shape
        offset_matrix = np.ones(matrix_shape, dtype=int) * profit_matrix.max()
        return offset_matrix - profit_matrix

    def _adjust_matrix_by_min_uncovered_num(self, result_matrix, covered_rows, covered_columns):
        elements = []
        for row_index, row in enumerate(result_matrix):
            if row_index not in covered_rows:
                for index, element in enumerate(row):
                    if index not in covered_columns:
                        elements.append(element)
        min_uncovered_num = min(elements)

        adjusted_matrix = result_matrix.copy()
        for row in covered_rows:
            adjusted_matrix[row] += min_uncovered_num
        for column in covered_columns:
            adjusted_matrix[:, column] += min_uncovered_num

        adjusted_matrix -= min_uncovered_num
        return adjusted_matrix

    def __find_matches(self, zero_locations):
        marked_rows = []
        marked_columns = []

        for index, row in enumerate(zero_locations):
            if np.sum(row) == 1:
                column_index, = np.where(row)
                marked_rows, marked_columns = self.__mark_rows_and_columns(marked_rows, marked_columns, index, column_index)

        for index, column in enumerate(zero_locations.T):
            if np.sum(column) == 1:
                row_index, = np.where(column)
                marked_rows, marked_columns = self.__mark_rows_and_columns(marked_rows, marked_columns, row_index, index)

        return marked_rows, marked_columns

    @staticmethod
    def __mark_rows_and_columns(marked_rows, marked_columns, row_index, column_index):
        if row_index not in marked_rows and column_index not in marked_columns:
            marked_rows.append(row_index)
            marked_columns.append(column_index)
        return marked_rows, marked_columns

    @staticmethod
    def select_arbitrary_match(zero_locations):
        rows, columns = np.where(zero_locations)
        zero_count = []
        for index, row in enumerate(rows):
            total_zeros = np.sum(zero_locations[row]) + np.sum(zero_locations[:, columns[index]])
            zero_count.append(total_zeros)

        indices = zero_count.index(min(zero_count))
        return np.array([rows[indices]]), np.array([columns[indices]])

    def __set_results(self, result_lists):
        for result in result_lists:
            row, column = result
            if row < self._maxRow and column < self._maxColumn:
                self._results.append((int(row), int(column)))

class CoverZeros:
    def __init__(self, matrix):
        self._zero_locations = (matrix == 0)
        self._shape = matrix.shape
        self._choices = np.zeros(self._shape, dtype=bool)
        self._marked_rows = []
        self._marked_columns = []
        self.__calculate()
        self._covered_rows = list(set(range(self._shape[0])) - set(self._marked_rows))
        self._covered_columns = self._marked_columns

    def get_covered_rows(self):
        return self._covered_rows

    def get_covered_columns(self):
        return self._covered_columns

    def __calculate(self):
        while True:
            self._marked_rows = []
            self._marked_columns = []

            for index, row in enumerate(self._choices):
                if not row.any():
                    self._marked_rows.append(index)

            if not self._marked_rows:
                return True

            num_marked_columns = self.__mark_new_columns_with_zeros_in_marked_rows()

            if num_marked_columns == 0:
                return True

            while self.__choice_in_all_marked_columns():
                num_marked_rows = self.__mark_new_rows_with_choices_in_marked_columns()
                if num_marked_rows == 0:
                    return True
                num_marked_columns = self.__mark_new_columns_with_zeros_in_marked_rows()
                if num_marked_columns == 0:
                    return True

            choice_column_index = self.__find_marked_column_without_choice()

            while choice_column_index is not None:
                choice_row_index = self.__find_row_without_choice(choice_column_index)

                new_choice_column_index = None
                if choice_row_index is None:
                    choice_row_index, new_choice_column_index = self.__find_best_choice_row_and_new_column(choice_column_index)
                    if new_choice_column_index is not None:
                        self._choices[choice_row_index, new_choice_column_index] = False

                self._choices[choice_row_index, choice_column_index] = True
                choice_column_index = new_choice_column_index

    def __mark_new_columns_with_zeros_in_marked_rows(self):
        num_marked_columns = 0
        for index, column in enumerate(self._zero_locations.T):
            if index not in self._marked_columns:
                if column.any():
                    row_indices, = np.where(column)
                    zeros_in_marked_rows = (set(self._marked_rows) & set(row_indices)) != set([])
                    if zeros_in_marked_rows:
                        self._marked_columns.append(index)
                        num_marked_columns += 1
        return num_marked_columns

    def __mark_new_rows_with_choices_in_marked_columns(self):
        num_marked_rows = 0
        for index, row in enumerate(self._choices):
            if index not in self._marked_rows:
                if row.any():
                    column_index, = np.where(row)
                    if column_index in self._marked_columns:
                        self._marked_rows.append(index)
                        num_marked_rows += 1
        return num_marked_rows

    def __choice_in_all_marked_columns(self):
        for column_index in self._marked_columns:
            if not self._choices[:, column_index].any():
                return False
        return True

    def __find_marked_column_without_choice(self):
        for column_index in self._marked_columns:
            if not self._choices[:, column_index].any():
                return column_index
        return None

    def __find_row_without_choice(self, choice_column_index):
        row_indices, = np.where(self._zero_locations[:, choice_column_index])
        for row_index in row_indices:
            if not self._choices[row_index].any():
                return row_index
        return None

    def __find_best_choice_row_and_new_column(self, choice_column_index):
        row_indices, = np.where(self._zero_locations[:, choice_column_index])
        for row_index in row_indices:
            column_indices, = np.where(self._choices[row_index])
            column_index = column_indices[0]
            if self.__find_row_without_choice(column_index) is not None:
                return row_index, column_index
        return row_indices[0], None


In [ ]:
import numpy as np

class TransportationProblem:

    def __init__(self, grid, supply, demand):
        self.grid = grid
        self.supply = supply
        self.demand = demand
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])
        self.allocations = [[0]*self.num_cols for _ in range(self.num_rows)]

        if sum(demand) < sum(supply):
          self.demand.append(sum(supply)-sum(demand))
          for i in range(len(grid)):
              grid[i].append(0)
        if sum(supply) < sum(demand):
            self.supply.append(sum(demand)-sum(supply))
            grid.append([0]*len(grid[0]))



    def check_degeneracy(self):
        allocated_cells = 0
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                if self.allocations[i][j] > 0:
                    allocated_cells += 1

        # Calculate m + n - 1
        required_allocations = self.num_rows + self.num_cols - 1
        if allocated_cells < required_allocations:
            return "Degenerate"
        else:
            return "Non-Degenerate"



    def NorthWest_corner_method(self):
        startR = 0  # starting row
        startC = 0  # starting column
        ans = 0
        supply_remaining = self.supply[:]
        demand_remaining = self.demand[:]

        while any(supply_remaining) and any(demand_remaining):

            allocated = min(supply_remaining[startR], demand_remaining[startC])
            ans += allocated * self.grid[startR][startC]

            self.allocations[startR][startC] = allocated

            supply_remaining[startR] -= allocated
            demand_remaining[startC] -= allocated

            if supply_remaining[startR] < demand_remaining[startC]:
                startR += 1
            else:
                startC += 1

        return ans, self.allocations



    def LeastCost_method(self):
        supply_remaining = self.supply[:]
        demand_remaining = self.demand[:]
        ans = 0

        while any(supply_remaining) and any(demand_remaining):
            min_cost = float('inf')
            min_row = -1
            min_col = -1

            for i in range(self.num_rows):
                for j in range(self.num_cols):
                    if supply_remaining[i] > 0 and demand_remaining[j] > 0:
                        if self.grid[i][j] < min_cost:
                            min_cost = self.grid[i][j]
                            min_row, min_col = i, j

            allocated = min(supply_remaining[min_row], demand_remaining[min_col])
            ans += allocated * self.grid[min_row][min_col]

            self.allocations[min_row][min_col] = allocated

            supply_remaining[min_row] -= allocated
            demand_remaining[min_col] -= allocated

        return ans, self.allocations




    def calculate_penalties(self, costs, supply_remaining, demand_remaining):
        row_penalties = []
        col_penalties = []

        # Row Penalties
        for i in range(self.num_rows):
            if supply_remaining[i] > 0:
                row_costs = sorted([costs[i][j] for j in range(self.num_cols) if demand_remaining[j] > 0])
                if len(row_costs) > 1:
                    row_penalties.append(row_costs[1] - row_costs[0])  # Penalty is the difference between the lowest two costs
                else:
                    row_penalties.append(float('inf'))  # If only one cost is left, penalty is infinite
            else:
                row_penalties.append(float('-inf'))  # If supply is exhausted, set penalty to -inf to ignore

        # Column Penalties
        for j in range(self.num_cols):
            if demand_remaining[j] > 0:
                col_costs = sorted([costs[i][j] for i in range(self.num_rows) if supply_remaining[i] > 0])
                if len(col_costs) > 1:
                    col_penalties.append(col_costs[1] - col_costs[0])
                else:
                    col_penalties.append(float('inf'))
            else:
                col_penalties.append(float('-inf'))

        return row_penalties, col_penalties



    def Vogels_approximation_method(self):
        ans = 0
        supply_remaining = self.supply[:]
        demand_remaining = self.demand[:]
        costs = self.grid

        while any(supply_remaining) and any(demand_remaining):
            row_penalties, col_penalties = self.calculate_penalties(costs, supply_remaining, demand_remaining)

            max_row_penalty = max(row_penalties)
            max_col_penalty = max(col_penalties)

            r_index = -1
            c_index = -1

            if max_row_penalty >= max_col_penalty:
                row_index = row_penalties.index(max_row_penalty)
                # To find min col. index
                min_cost = float('inf')
                min_col_index = -1
                for j in range(self.num_cols):
                    if demand_remaining[j] > 0:  # Check if demand is still available for this row
                        if costs[row_index][j] < min_cost:
                            min_cost = costs[row_index][j]
                            min_col_index = j
                r_index = row_index
                c_index = min_col_index


            else:
                col_index = col_penalties.index(max_col_penalty)
                min_cost = float('inf')
                min_row_index = -1
                # To find min row. index
                for i in range(self.num_rows):
                    if supply_remaining[i] > 0:  # Check if supply is still available for this row
                        if costs[i][col_index] < min_cost:
                            min_cost = costs[i][col_index]
                            min_row_index = i
                r_index = min_row_index
                c_index = col_index

            allocated = min(supply_remaining[r_index], demand_remaining[c_index])
            ans += allocated * costs[r_index][c_index]

            self.allocations[r_index][c_index] = allocated

            supply_remaining[r_index] -= allocated
            demand_remaining[c_index] -= allocated

        return ans, self.allocations




    def getBasicVariablePosition(self):
        position = []
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                if self.allocations[i][j] > 0:
                    position.append((i,j))
        return position


    def getBasicVariable(self):
        basic = []
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                if self.allocations[i][j] > 0:
                    basic.append(((i,j),self.allocations[i][j]))
        return basic


    def get_uv(self, BasicFeasibleSolution):
        u = [None] * self.num_rows
        v = [None] * self.num_cols
        u[0] = 0  # Start by assigning u1 = 0

        basicFeasibleSolutionArr = BasicFeasibleSolution.copy()

        while len(basicFeasibleSolutionArr) > 0:
            for index, basicVariable in enumerate(basicFeasibleSolutionArr):
                i, j = basicVariable[0]
                if u[i] is None and v[j] is None:
                    continue
                cost = self.grid[i][j]
                if u[i] is None:
                    u[i] = cost - v[j]
                else:
                    v[j] = cost - u[i]
                basicFeasibleSolutionArr.pop(index)
                break
        return (u, v)


    def get_delta(self,BasicFeasibleSolution,u,v):
        # Calculate d_ij = U_i + V_j - C_ij for all unallocated cells
        W = []
        #print(u,v)
        for i, row in enumerate(self.grid):
            for j, cost in enumerate(row):
                isBasic = any(p[0] == i and p[1] == j for p, v in BasicFeasibleSolution)
                if not isBasic:  # Only consider non-basic variables
                    W.append(((i, j), u[i] + v[j] - cost))
        return W



    def isOptimal(self, delta):
        for p, v in delta:
            if v > 0:
                return False
        return True

    def getEnteringVariablePosition(self,delta):
        d = delta.copy()
        d.sort(key = lambda w: w[1])
        return d[-1][0]


    def getPossibleNextCells(self, loop, notVisited):
        lastCell = loop[-1]
        rowCells = [n for n in notVisited if n[0] == lastCell[0]]
        columnCells = [n for n in notVisited if n[1] == lastCell[1]]
        if len(loop) < 2:
            return rowCells + columnCells
        else:
            previousCell = loop[-2]
            rowMove = previousCell[0] == lastCell[0]
            if rowMove:
                return columnCells
            return rowCells


    def get_loop(self, basicVariablePositions, enteringVariablePosition):
        def inner(loop):
            if len(loop) > 3:
                canBeClosed = len(self.getPossibleNextCells(loop, [enteringVariablePosition])) == 1
                if canBeClosed:
                    return loop

            notVisited = list(set(basicVariablePositions) - set(loop))
            possibleNextCells = self.getPossibleNextCells(loop, notVisited)
            for nextCell in possibleNextCells:
                newLoop = inner(loop + [nextCell])
                if newLoop:
                    return newLoop
        return inner([enteringVariablePosition])



    def loopPivoting(self, BasicFeasibleSolution, loop):
        evenCells = loop[0: :2]
        oddCells = loop[1: :2]

        getBasicVariables = lambda pos: next(v for p, v in BasicFeasibleSolution if p == pos)
        leavingPosition = sorted(oddCells, key=getBasicVariables)[0]
        leavingValue = getBasicVariables(leavingPosition)

        newBasicFeasibleSolution = []
        for p, v in [basicVariable for basicVariable in BasicFeasibleSolution if basicVariable[0] != leavingPosition] + [(loop[0], 0)]:
            if p in evenCells:
                v += leavingValue
            elif p in oddCells:
                v -= leavingValue
            newBasicFeasibleSolution.append((p, v))

        return newBasicFeasibleSolution


    def MODI_method(self, BasicFeasibleSolution):
        u,v = self.get_uv(BasicFeasibleSolution)
        delta = self.get_delta(BasicFeasibleSolution, u, v)
        optimal = self.isOptimal(delta)
        #print(delta)

        if optimal:
            return BasicFeasibleSolution
        else:
            enteringVariablePosition = self.getEnteringVariablePosition(delta)
            loop = self.get_loop([p for p, v in BasicFeasibleSolution], enteringVariablePosition)
            return self.MODI_method(self.loopPivoting(BasicFeasibleSolution, loop))


    def getTotalCost(self, ans):
        totalCost = 0
        for i, row in enumerate(self.grid):
            for j, cost in enumerate(row):
                totalCost += cost * ans[i][j]
        return totalCost


    def Optimal_solution(self):
        ans, alloc = self.Vogels_approximation_method()
        BasicFeasibleSolution = self.getBasicVariable()
        degeneracy = self.check_degeneracy()
        if degeneracy=="Degenerate":
            return

        solutionSet = self.MODI_method(BasicFeasibleSolution)
        opt = np.zeros((self.num_rows, self.num_cols))
        for (i, j), v in solutionSet:
            opt[i][j] = int(v)

        return ans, np.array(alloc), self.getTotalCost(opt), opt

In [ ]:
import gradio as gr

def solve_hungarian(maximize, matrix):
    try:
        # Check if the input is not empty
        if not matrix.strip():
            raise ValueError("Input matrix cannot be empty.")

        # Convert matrix input to a 2D list
        rows = matrix.split(';')

        # Check for empty rows and strip whitespace
        matrix = [list(map(int, row.strip().split(','))) for row in rows if row.strip()]

        # Check for any empty rows after processing
        if any(len(row) == 0 for row in matrix):
            raise ValueError("One or more rows in the matrix are empty.")

        print("Parsed matrix:", matrix)  # Debugging line

        hungarian = Hungarian()
        hungarian.input_matrix(matrix, is_profit_matrix=maximize)
        hungarian.calculate()

        return {
            "Total": hungarian._totalPotential,  # Total potential or cost
            "Results": hungarian._results        # Results from the Hungarian method
        }

    except ValueError as e:
        return f"ValueError: {e}"
    except Exception as e:
        return f"Error processing input: {e}"




def main(matrix, maximize):
    result = solve_hungarian(maximize, matrix)

    if isinstance(result, dict):
      output_label = "Hungarian Output"
      output_value = (f"Total Cost/Potential: {result['Total']}\n"
                      f"Results:\n{result['Results']}")

      return output_label,output_value  # Ensure to return both the label and value

    else:
        return "Error", result  # Return the error message


# Define the interface
iface = gr.Interface(
    fn=main,
    inputs=[
        gr.Textbox(label="Enter cost Matrix (comma-separated values per row, e.g., '1,2,3;4,5,6')"),
        gr.Checkbox(label="Maximize (Only for Hungarian)")
    ],
    outputs=["text","text"],
    title="Assignment Problem",
    description="Enter the input cost matrix for to solve using hungarian method"
                "Check 'Maximize' to set Hungarian to Maximization mode."
)

if __name__ == "__main__":
    iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd763618c98eb46e09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def solve_transportation(IBFS_method, optimum, matrix, supply_input, demand_input):
    try:
        # Check if the input is not empty
        if not matrix.strip():
            raise ValueError("Input matrix cannot be empty.")

        # Convert matrix input to a 2D list
        rows = matrix.split(';')

        # Check for empty rows and strip whitespace
        matrix = [list(map(int, row.strip().split(','))) for row in rows if row.strip()]

        # Check for any empty rows after processing
        if any(len(row) == 0 for row in matrix):
            raise ValueError("One or more rows in the matrix are empty.")

        print("Parsed matrix:", matrix)  # Debugging line

        supply = list(map(int, supply_input.strip().split(',')))
        demand = list(map(int, demand_input.strip().split(',')))

        # Validate dimensions
        if len(supply) != len(matrix):
            raise ValueError("The number of supply entries must match the number of rows in the matrix.")
        if len(demand) != len(matrix[0]):
            raise ValueError("The number of demand entries must match the number of columns in the matrix.")

        if IBFS_method == "North-West corner method":
            IBFS = TransportationProblem(matrix, supply, demand)
            solution, allocation = IBFS.NorthWest_corner_method()
        elif IBFS_method == "Least Cost method":
            IBFS = TransportationProblem(matrix, supply, demand)
            solution, allocation = IBFS.LeastCost_method()
        elif IBFS_method == "Vogal's Approximation Method":
            IBFS = TransportationProblem(matrix, supply, demand)
            solution, allocation = IBFS.Vogels_approximation_method()


        if optimum:
          OBFS = TransportationProblem(matrix, supply, demand)
          initial_solution, initial_allocation, optimal_solution, optimal_allocation = OBFS.Optimal_solution()
          return {
            "Solution": solution,                      # IBFS sol
            "Allocation": np.array(allocation),        # IBFS alloc
            "Initial Solution": initial_solution,      # Initial cost from the transportation method
            "Initial Allocation": initial_allocation,  # Allocated values
            "Optimal Solution": optimal_solution,      # Optimal cost from the transportation method
            "Optimal Allocation": optimal_allocation   # Allocated values
          }

        else:
          return {
              "Solution": solution,
              "Allocation": np.array(allocation)
          }

    except ValueError as e:
        return f"ValueError: {e}"
    except Exception as e:
        return f"Error processing input: {e}"



def main(IBFS_method, optimum, matrix, supply, demand):

    result = solve_transportation(IBFS_method, optimum, matrix, supply, demand)

    # Check if result is a string (error message) or a dictionary
    if isinstance(result, dict):
      if optimum:
          output_label = "Transportation Output"
          output_value = (f"{IBFS_method}\n"
                          f"Initial Basic Feasible Solution: {result['Solution']}\n"
                          f"Allocation:\n{result['Allocation']}\n\n"
                          "**********************************************************************\n\n"
                          f"IBFS (Vogal's Approximation): {result['Initial Solution']}\n"
                          f"Allocation:\n{result['Initial Allocation']}\n\n"
                          f"Optimal Solution: {result['Optimal Solution']}\n"
                          f"Allocation:\n{result['Optimal Allocation']}")
      else:
          output_label = "Transportation Output"
          output_value = (f"{IBFS_method}\n"
                          f"Initial Basic Feasible Solution: {result['Solution']}\n"
                          f"Allocation:\n{result['Allocation']}\n\n")

      return output_label,output_value  # Ensure to return both the label and value

    else:
        return "Error", result  # Return the error message


# Define the interface
iface = gr.Interface(
    fn=main,
    inputs=[
        gr.Radio(["North-West corner method", "Least Cost method", "Vogal's Approximation Method"], label="Select a method to find Initial Basic Feasible solution"),
        gr.Checkbox(label="Find Optimal solution (Using Vogal's Appriximation and MODI's method)"),
        gr.Textbox(label="Enter cost Matrix (comma-separated values per row, e.g., '1,2,3;4,5,6')"),
        gr.Textbox(label="Enter supply w.r.t each row of cost matrix (comma-separated values, e.g., '1,2,3')"),
        gr.Textbox(label="Enter demand w.r.t each column of cost matrix (comma-separated values, e.g., '1,2,3')")
    ],
    outputs=["text", "text"],
    title="Transportation problem",
    description="Enter the cost matrix and the corresponding supply and demand to solve the transportation problem"
                "Check 'Find optimal Solution to get the optimum solution using IBFS-Vogal's and OBFS-MODI's method."
)

if __name__ == "__main__":
    iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://596a07d04b5409f42c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
